![Wherobots Logo](https://github.com/SeanLikesData/public_assets/blob/main/header-logo.png?raw=true)
# Wildfire Risk Assessment Using Wherobots

This tutorial demonstrates how to build a comprehensive wildfire risk assessment model using Wherobots' spatial analytics capabilities. We'll combine USDA Forest Service burn probability data with building footprints and emergency service locations to identify high-risk areas in Southern California.

![Wildfire Image](https://github.com/SeanLikesData/public_assets/blob/main/ca_camp_fire.png?raw=true)
*2018 Camp Fire in California*

## Overview

This analysis will:
1. Create uniform analysis units using H3 hexagons
2. Calculate risk factors including:
   - Burn probability from USDA Forest Service models
   - Building density from Overture Maps data
   - Distance to nearest fire station
3. Combine these factors into a weighted risk score
4. Identify statistically significant clusters of high-risk areas

## Data Sources

- **Burn Probability Data**: USDA Forest Service [FSim Fire Probability Dataset](https://www.fs.usda.gov/rds/archive/catalog/RDS-2020-0016-2)
- **Overture Maps Foundation**:
  - Building footprints
  - Fire station locations

## Technical Concepts We'll Cover

- Loading and processing large raster datasets
- Performing spatial joins at scale
- Using H3 spatial indexing
- Computing zonal statistics
- Conducting hot spot analysis
- Creating interactive visualizations

## Prerequisites

- Basic understanding of SQL and Python
- Familiarity with geospatial concepts
- Access to a Wherobots account

# 2. Burn Probability


## Sedona Context

When using Wherobots the first step is to create the `Context`. This context represents the connection to the cluster. The first time we set the config and create the `Context` all of the nodes in the cluster are brought on line


In [2]:
%%time
from sedona.spark import *
from pyspark.sql.functions import col, count, when, expr,lit,explode,broadcast, desc
from datetime import datetime
start = datetime.now()

config = SedonaContext.builder(). \
    config("spark.hadoop.fs.s3a.bucket.wherobots-examples.aws.credentials.provider","org.apache.hadoop.fs.s3a.AnonymousAWSCredentialsProvider"). \
    getOrCreate()

sedona = SedonaContext.create(config)
sedona.sparkContext.setLogLevel("ERROR")

CPU times: user 6.69 ms, sys: 0 ns, total: 6.69 ms
Wall time: 1.62 s


### Loading the Burn Probability Raster Data
We'll start by loading our key input dataset - a burn probability raster from the USDA Forest Service. This raster represents the annual probability of a wildfire burning in any given location. Our analysis will focus on Southern California, specifically excluding parts of Mexico and the Pacific Ocean using a polygon boundary defined in WKT (Well-Known Text) format.

![Burn Probability Raster Image](https://github.com/SeanLikesData/public_assets/blob/main/ca-fire.jpg?raw=true)

The code below:
1. Specifies the S3 path to our burn probability GeoTIFF
2. Defines an exclusion area for our study area as a MultiPolygon in WKT format 
3. Creates a dataframe by reading the raster file

### Tiling and Processing the Burn Probability Raster

To efficiently process our large burn probability raster, we need to break it into smaller 256x256 pixel tiles. This tiling strategy allows for distributed processing. 
This process used to require extra steps, but we have recently built it inot our read operation.

We then run a SQL query to generate summary stats using `RS_SUMMARYSTATS()`. This creates a tiled dataset with summary statistics for each 256x256 section of our study area, which we'll later combine with other risk factors. 

In [3]:
%%time
## Wall time: 14.8 s (LARGE)
analysis_start = datetime.now()
burn_probability = "s3://wherobots-examples/data/examples/SDSC/BP-clip-reblock-4326.tif"
exclusion_wkt = "MultiPolygon (((-119.22425369 31.67241078, -119.03742835 30.92881246, -117.00686077 31.26615639, -117.24927621 32.01643187, -117.36169794 32.50929768, -117.40697765 32.65986814, -117.45368807 32.79856211, -117.50463868 32.9832308, -117.59444643 33.13521645, -117.81002531 33.3313064, -118.16274878 33.51473174, -118.44751165 33.54860864, -118.54278981 33.70994242, -118.62402192 33.86683156, -118.77557516 33.93388227, -118.86933137 33.98250615, -118.89850234 33.97769324, -119.25883116 33.91883788, -119.84475305 33.82227912, -119.35342742 32.16682393, -119.28359676 31.88814932, -119.22425369 31.67241078)))"

burn_probability_df = (sedona.read.format("raster")
                             .option("retile", True)
                             .option("tileWidth",256)
                             .option("tileHeight",256)
                             .load(burn_probability)
                             .where(f"""ST_INTERSECTS(RS_ENVELOPE(rast), ST_GEOMFROMWKT('{exclusion_wkt}')) = False"""))

burn_probability_df.createOrReplaceTempView("burn_probability")

burn_probability_df_tiled = sedona.sql(f""" 
SELECT
    *, 
    RS_SUMMARYSTATS(rast,'max') as max_bp,
    RS_SUMMARYSTATS(rast,'mean') as mean_bp, 
    RS_SUMMARYSTATS(rast,'min') as min_bp
from burn_probability 
where  RS_SUMMARYSTATS(rast, 'count') > 0
"""
                                )
burn_probability_df_tiled.createOrReplaceTempView("burn_probability_tiled")
burn_probability_df_tiled.count()

CPU times: user 30.2 ms, sys: 0 ns, total: 30.2 ms
Wall time: 16.5 s


991

<img src="./assets/exclusion.png" width="600">
<i>Visual of our exclusion area</i>

### Examining the Tiled Burn Probability Data

Let's look at a sample of the tiled data to understand the structure and values we're working with. For each tile (referenced by its x,y coordinates), we'll see:

- mean_bp: The average burn probability in the tile
- max_bp: The highest burn probability value in the tile  
- min_bp: The lowest burn probability value in the tile

These probability values are between 0 and 1, where higher values indicate a greater chance of wildfire. The values are in scientific notation - for example, 1.67E-4 means 0.000167 or about a 0.0167% annual probability of burning.

In [4]:
%%time
burn_probability_df_tiled.select("x", "y","mean_bp","max_bp","min_bp").show(10,truncate=False)

+---+---+---------------------+---------------------+---------------------+
|x  |y  |mean_bp              |max_bp               |min_bp               |
+---+---+---------------------+---------------------+---------------------+
|34 |1  |0.0015406080794939543|0.00225593033246696  |3.5467493580654263E-4|
|26 |16 |0.0069544134670318294|0.034708503633737564 |0.0                  |
|39 |11 |4.4839653100307435E-4|0.0016230391338467598|0.0                  |
|27 |13 |0.006816646234578463 |0.014586788602173328 |0.0                  |
|40 |3  |4.828258165987531E-4 |7.410030229948461E-4 |9.907162166200578E-5 |
|20 |18 |0.0024328396763276783|0.014860003255307674 |0.0                  |
|6  |10 |1.4239575725917942E-5|5.303890211507678E-4 |0.0                  |
|6  |13 |4.249758735049285E-5 |0.0014025315176695585|0.0                  |
|36 |6  |9.380286052925942E-4 |0.0013458288740366697|0.0                  |
|23 |18 |0.010771378579811093 |0.024730166420340538 |0.0                  |
+---+---+---

# 3. H3 Hexagonal Grid Units

### Creating H3 Hexagonal Grid Units

To analyze wildfire risk systematically, we'll divide our study area into uniform hexagonal cells using Uber's H3 spatial indexing system. We use H3 level 6 hexagons, which are approximately 36 km² each - large enough to capture meaningful patterns but small enough for detailed analysis.

The SQL query below performs several steps:
1. Generates H3 cell IDs that cover our burn probability raster tiles 
2. Converts these IDs into geometric hexagons
3. Filters out hexagons that intersect with our exclusion area
4. Caches the result for faster subsequent operations

The resulting hexagonal grid will serve as our fundamental analysis units for calculating risk factors. We repartition the data into 15 partitions to optimize parallel processing across our cluster.

In [5]:
%%time
# 9.79 s
h3_level=6

hex_df = sedona.sql(f""" 
    WITH 
        h3_ids AS (
            SELECT 
               DISTINCT EXPLODE(ST_H3CellIDs(RS_Envelope(rast), {h3_level}, true)) AS h3 
            FROM 
                burn_probability_tiled),
        
        exp_h3 as (
            SELECT 
                h3,
                EXPLODE (ST_H3ToGeom(ARRAY(h3))) AS geometry
            FROM 
                h3_ids)
    
    SELECT * FROM exp_h3 
    WHERE ST_INTERSECTS(geometry, ST_GEOMFROMWKT('{exclusion_wkt}')) = False
""").repartition(15).cache()

hex_df.createOrReplaceTempView("mapping_units")
hex_df.count()

CPU times: user 21.5 ms, sys: 1.27 ms, total: 22.8 ms
Wall time: 9.52 s


1456

### Visualizing Our Analysis Framework

Let's create an interactive map to visualize two key components of our analysis setup:

1. The H3 hexagonal grid we created earlier (our analysis units)
2. The 256x256 burn probability tiles

This visualization helps us verify:
- The coverage and size of our H3 hexagons
- The distribution of our raster tiles
- How these two spatial frameworks overlap
- That our exclusion area is properly removed

The map creation involves:
1. Creating a new base map using SedonaKepler
2. Adding the H3 hexagonal grid layer
3. Adding the burn probability tiles after:
  - Converting each tile to its envelope (bounding box) using RS_Envelope
  - Calculating the area of each tile using ST_AREA

This visual check helps ensure our spatial frameworks are properly aligned before proceeding with the analysis.

In [6]:
burn_probability_df_tiled.printSchema()

root
 |-- rast: raster (nullable = true)
 |-- x: integer (nullable = true)
 |-- y: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- max_bp: double (nullable = true)
 |-- mean_bp: double (nullable = true)
 |-- min_bp: double (nullable = true)



In [7]:
tiledMap = SedonaKepler.create_map()
SedonaKepler.add_df(tiledMap, hex_df, name="h3")
SedonaKepler.add_df(tiledMap, burn_probability_df_tiled.withColumn("tile", expr("RS_Envelope(rast)")).withColumn("area",expr("ST_AREA(tile)")), name="tiles")
tiledMap

KeplerGl(data={'h3': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, …

# 4. Calculate Building Density

In this section we will use the Overture Maps Foundation Buildings dataset to calculate building density metric for our composit risk score.  


In [8]:
hex_df.printSchema()

root
 |-- h3: long (nullable = true)
 |-- geometry: geometry (nullable = true)



### Creating a Single Study Area Geometry

Let's combine all our H3 hexagons into a single geometry that represents our complete study area. This unified geometry will be useful for:

1. Spatial filtering of our building and fire station data
2. Ensuring consistent coverage across datasets
3. Reducing processing time by quickly filtering out irrelevant features

We use ST_Union_Aggr to dissolve all hexagon boundaries, creating one multipolygon. The .collect()[0].geom syntax retrieves this geometry from the distributed dataframe into a single Python object we can reference in subsequent queries.

This operation essentially creates the boundary of our analysis area by unifying the geometries of all our H3 hexagons that don't intersect with our exclusion areas.

In [9]:
%%time

aoi_geom = sedona.sql("""SELECT ST_Union_Aggr(geometry) AS geom
    FROM mapping_units""").collect()[0].geom

CPU times: user 0 ns, sys: 4.18 ms, total: 4.18 ms
Wall time: 817 ms


### Loading Building Data for the Study Area

Now we'll extract building footprints from the Overture Maps database that fall within our study area. Instead of processing all buildings in the database, we use spatial filtering to get only the buildings we need:

1. We read from the Wherobots-hosted Overture Maps building layer
2. We select just the building ID and geometry
3. We use ST_WITHIN to keep only buildings that fall inside our study area geometry

This spatial filter significantly reduces the data we need to process in subsequent steps. The ST_WITHIN function ensures we only get buildings completely contained within our study area boundary, as opposed to buildings that might just intersect it.

The Wall time of 10.3 seconds shows how Sedona efficiently handles this spatial query across what could be millions of building footprints.

In [10]:
%%time
#Wall time: 10.3 s
aoi_buildings_df = sedona.sql(
f"""
SELECT 
    ID, geometry
FROM 
    wherobots_open_data.overture_maps_foundation.buildings_building
WHERE 
    ST_WITHIN(geometry,ST_GEOMFROMWKT('{aoi_geom}')) """)

CPU times: user 0 ns, sys: 2.99 ms, total: 2.99 ms
Wall time: 4.16 s


In [11]:
%%time
#aoi_buildings_df = aoi_buildings_df.repartition(100)
aoi_buildings_df.createOrReplaceTempView('aoi_buildings')
aoi_buildings_df.count()

CPU times: user 6.4 ms, sys: 5.44 ms, total: 11.8 ms
Wall time: 7.85 s


8324176

### Calculating Building Density Per Hexagon

For each H3 hexagon in our study area, we'll calculate two key metrics about buildings:

1. Building Count: The total number of buildings in each hexagon
2. Building Density: The ratio of total building footprint area to hexagon area

The SQL query joins our H3 hexagons with buildings using a spatial relationship. We count a building as being in a hexagon if its centroid (center point) falls within the hexagon. This avoids double-counting buildings that might straddle hexagon boundaries.

For density calculation:
- Numerator: Sum of all building footprint areas in the hexagon (using ST_AreaSpheroid for accurate area calculations on a spheroid)
- Denominator: Area of the hexagon itself
- Result: A ratio between 0 and 1, where higher values indicate more built-up areas

This building density metric will be an important factor in our wildfire risk assessment, as areas with higher building density typically represent greater potential loss in a fire event.

In [12]:
%%time
building_density_df = sedona.sql(
"""
SELECT 
    t1.h3,
    t1.geometry,
    COUNT(t2.ID) building_count,
    (SUM(ST_AreaSpheroid(t2.geometry))/ST_AreaSpheroid(t1.geometry)) building_density
FROM 
    mapping_units t1
 JOIN
    aoi_buildings t2
ON
    ST_INTERSECTS(t1.geometry,ST_CENTROID(t2.geometry))
GROUP BY
    t1.h3,
    t1.geometry
""" 
)
building_density_df.createOrReplaceTempView("building_density")

CPU times: user 1.49 ms, sys: 0 ns, total: 1.49 ms
Wall time: 41.5 ms


In [13]:
%%time
#Wall time: 2min 24s
#Wall time: 39min 23s
building_density_df = building_density_df.cache()

building_density_df.show(20)

+------------------+--------------------+--------------+--------------------+
|                h3|            geometry|building_count|    building_density|
+------------------+--------------------+--------------+--------------------+
|604214660782096383|POLYGON ((-117.52...|           212|0.001042046518309...|
|604214731917492223|POLYGON ((-118.64...|          3340| 0.01747436372001797|
|604214681854279679|POLYGON ((-117.04...|          3676|0.022689137848988142|
|604755114636345343|POLYGON ((-116.10...|            31|9.219054189333367E-5|
|604214931365036031|POLYGON ((-117.22...|         16238| 0.10162872500996421|
|604214694470746111|POLYGON ((-118.43...|          5167|   0.027380507426555|
|604215049745072127|POLYGON ((-116.45...|          1091| 0.00522290871543986|
|604214698094624767|POLYGON ((-118.46...|          6149|  0.1371750712106065|
|604215001158254591|POLYGON ((-118.07...|         12563| 0.10546007676249752|
|604755113428385791|POLYGON ((-116.56...|            21|4.951939

### Combining Building Metrics with Burn Probability Data

Now we'll merge our building metrics with the burn probability data to create a more comprehensive picture of risk. For each H3 hexagon, we combine:

1. Previously calculated building metrics:
  - Building count 
  - Building density

2. Burn probability statistics from all raster tiles that intersect each hexagon:
  - Maximum burn probability (worst case)
  - Minimum burn probability (best case)
  - Average burn probability (typical case)
  - Range of burn probabilities (variability within hexagon)

The spatial join uses ST_INTERSECTS with RS_ENVELOPE to find which raster tiles overlap each hexagon. We aggregate these tiles' statistics to get a complete picture of burn probability for each hexagon.

The query takes about a minute to run, which is reasonable given the spatial complexity. We cache the results since we'll use them multiple times in subsequent analysis steps.

In [14]:
%%time
#Wall time: 1min 1s
max_burn_df = sedona.sql(f"""

SELECT
    t1.h3,
    t1.geometry,
    t1.building_count,
    t1.building_density,
    MAX(t2.max_bp) as max_bp, 
    MIN(t2.min_bp) as min_bp,
    AVG(t2.mean_bp) as mean_bp,
    MAX(t2.max_bp) - MIN(t2.min_bp) AS range_bp

FROM 
    building_density t1
 JOIN
  burn_probability_tiled t2
ON ST_INTERSECTS(t1.geometry,RS_ENVELOPE(t2.rast))
GROUP BY 
1,2,3,4


""")
max_burn_df = max_burn_df.cache()
max_burn_df.createOrReplaceTempView("max_burn")
max_burn_df.show()

+------------------+--------------------+--------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                h3|            geometry|building_count|    building_density|              max_bp|              min_bp|             mean_bp|            range_bp|
+------------------+--------------------+--------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|604214659574136831|POLYGON ((-117.33...|           888|0.007746056266217991| 0.10265446454286575|                 0.0| 0.05161579037182323| 0.10265446454286575|
|604214647091888127|POLYGON ((-116.77...|           642|0.002961006118760...| 0.09019006043672562|                 0.0|0.039032730602953654| 0.09019006043672562|
|604215086789165055|POLYGON ((-116.40...|           111|  2.9500974982111E-4|0.050787221640348434|                 0.0|0.006850700174079933|0.050787221640348434|
|604215039141871615|POLYGON 

In [15]:
building_density_df.unpersist()

DataFrame[h3: bigint, geometry: udt, building_count: bigint, building_density: double]

In [16]:
max_burn_df.printSchema()

root
 |-- h3: long (nullable = true)
 |-- geometry: geometry (nullable = true)
 |-- building_count: long (nullable = false)
 |-- building_density: double (nullable = true)
 |-- max_bp: double (nullable = true)
 |-- min_bp: double (nullable = true)
 |-- mean_bp: double (nullable = true)
 |-- range_bp: double (nullable = true)



# 5. KNN join 
We extract all the fire stations by primary catagory and using our AOI for spatial filter pushdown. 


### Loading Fire Station Locations

Another crucial factor in wildfire risk assessment is proximity to emergency services. We'll extract fire station locations from the Overture Maps places dataset, which includes various point-of-interest categories.

The query:
1. Reads from Overture's places table (2024-07 release)
2. Filters to include only locations categorized as fire departments
3. Uses ST_WITHIN to select only fire stations within our study area
4. Creates a temporary view for use in subsequent proximity analysis

When assessing wildfire risk, areas further from fire stations may face longer response times during an emergency, potentially increasing their risk level. In our next steps, we'll calculate distances from each hexagon to the nearest fire station.

Note: The query shows Overture's hierarchical category system in action - instead of searching for an exact string match, we're checking the 'primary' category field for 'fire_department'.

In [17]:
%%time
fire_stateion_df = sedona.sql(f"""
    SELECT * FROM 
        wherobots_open_data.overture_maps_foundation.places_place  
    WHERE 
        categories.primary IN ('fire_department')
    AND
        ST_WITHIN(geometry,ST_GEOMFROMWKT('{aoi_geom}'))
""")
fire_stateion_df.count()
fire_stateion_df.createOrReplaceTempView("fire_stations")

CPU times: user 3.23 ms, sys: 485 μs, total: 3.72 ms
Wall time: 1.6 s


### Calculating Distance to Nearest Fire Station

For each hexagon in our study area, we'll identify the closest fire station and calculate the distance to it. This creates another key risk factor - areas further from fire stations may face longer emergency response times.

The query uses several spatial functions:
1. ST_AKNN: A spatial K-Nearest Neighbor join that finds the closest fire station to each hexagon (K=1)
2. ST_DISTANCESPHERE: Calculates the true spherical distance between each hexagon and its nearest fire station
3. ST_CENTROID: Gets the center point of each hexagon
4. ST_MAKELINE: Creates a line connecting each hexagon center to its nearest fire station (useful for visualization)

The output includes:
- SOURCE_GEOM: The hexagon geometry
- h3: The hexagon ID
- sink_GEOM: The fire station location
- DISTANCE: Distance in meters to nearest station
- LINE: A visual connection between hexagon and station

This distance metric will be an important factor in our composite risk score, as it directly relates to emergency response capabilities.

In [18]:
%%time
df_knn_join = sedona.sql("""
    SELECT
        source.GEOMETRY AS SOURCE_GEOM,
        SOURCE.h3 AS h3,
        sink.GEOMETRY AS sink_GEOM,
        ST_DISTANCESPHERE(source.GEOMETRY, sink.GEOMETRY) AS DISTANCE,
        ST_MAKELINE(ST_CENTROID(source.GEOMETRY), sink.GEOMETRY) AS LINE
    FROM max_burn source

    JOIN fire_stations sink ON ST_AKNN(source.GEOMETRY, sink.GEOMETRY, 1, FALSE)
""")


CPU times: user 1.17 ms, sys: 0 ns, total: 1.17 ms
Wall time: 22.5 ms


### Cleaning Up the Nearest Fire Station Results

The KNN join might produce some duplicate results when hexagons are equidistant from multiple fire stations. We'll clean up these results in two steps:

1. Create a dataset with just one fire station per hexagon:
  - Use dropDuplicates on SOURCE_GEOM to keep just one result per hexagon
  
2. Join this unique set back to our original results:
  - Ensures we maintain all our calculated fields
  - Keeps our data structure consistent
  - Removes any unwanted duplicates

We cache both the unique and joined results since we'll use them multiple times in our risk calculations. The final count tells us how many hexagon-to-fire-station relationships we'll be working with.

This cleaned dataset gives us a one-to-one relationship between hexagons and their nearest fire stations, which is what we want for our risk assessment.

In [19]:
%%time
# Wall time: 1min 56s
# Select N unique QID rows
df_unique_qid = df_knn_join.dropDuplicates(["SOURCE_GEOM"])

# Perform an inner join to get all rows from join_df that have QIDs in unique_qid_df
df_related_rows = df_knn_join.join(df_unique_qid, on="h3", how="inner").select(df_knn_join["*"])

df_unique_qid.cache()
df_related_rows.cache()
df_related_rows.createOrReplaceTempView("fire_station_distance")
df_related_rows.count()

These filters will be applied to the object side reader before the KNN join is executed. 
If you intend to apply the filters after the KNN join, please ensure that you materialize the KNN join results before applying the filters. 
For example, you can use the following approach:

Scala Example:
val knnResult = knnJoinDF.cache()
val filteredResult = knnResult.filter(condition)

SQL Example:
CREATE OR REPLACE TEMP VIEW knnResult AS
SELECT * FROM (
  -- Your KNN join SQL here
) AS knnView
CACHE TABLE knnResult;
SELECT * FROM knnResult WHERE condition;
These filters will be applied to the object side reader before the KNN join is executed. 
If you intend to apply the filters after the KNN join, please ensure that you materialize the KNN join results before applying the filters. 
For example, you can use the following approach:

Scala Example:
val knnResult = knnJoinDF.cache()
val filteredResult = knnResult.filter(condition)

SQL Example:
CREATE OR REPLACE TEMP VIEW knnResult AS
SELECT * FR

These filters will be applied to the object side reader before the KNN join is executed. 
If you intend to apply the filters after the KNN join, please ensure that you materialize the KNN join results before applying the filters. 
For example, you can use the following approach:

Scala Example:
val knnResult = knnJoinDF.cache()
val filteredResult = knnResult.filter(condition)

SQL Example:
CREATE OR REPLACE TEMP VIEW knnResult AS
SELECT * FROM (
  -- Your KNN join SQL here
) AS knnView
CACHE TABLE knnResult;
SELECT * FROM knnResult WHERE condition;


CPU times: user 15.1 ms, sys: 678 μs, total: 15.8 ms
Wall time: 7.66 s


1101

In [20]:
df_unique_qid.show(5)

+--------------------+------------------+--------------------+------------------+--------------------+
|         SOURCE_GEOM|                h3|           sink_GEOM|          DISTANCE|                LINE|
+--------------------+------------------+--------------------+------------------+--------------------+
|POLYGON ((-118.53...|604214731514839039|POINT (-118.53636...|1756.1248502033416|LINESTRING (-118....|
|POLYGON ((-116.01...|604755134769004543|POINT (-115.99441...|15959.845548939582|LINESTRING (-116....|
|POLYGON ((-118.21...|604214997937029119|POINT (-118.24752...|3490.9830839994133|LINESTRING (-118....|
|POLYGON ((-116.52...|604215083836375039|POINT (-116.44805...| 9005.413226266133|LINESTRING (-116....|
|POLYGON ((-116.26...|604215076051746815|POINT (-116.42244...|14794.136559968048|LINESTRING (-116....|
+--------------------+------------------+--------------------+------------------+--------------------+
only showing top 5 rows



In [21]:
# create map for the results
map_view = SedonaKepler.create_map(df_unique_qid.select('SOURCE_GEOM'), name="SOURCE")
SedonaKepler.add_df(map_view, df=df_related_rows.select('SINK_GEOM', 'DISTANCE').withColumnRenamed("SINK_GEOM", "geometry"), name="SINK")
SedonaKepler.add_df(map_view, df=df_related_rows.select('LINE', 'DISTANCE').withColumnRenamed("LINE", "geometry"), name="KNN LINES")

# show the map
map_view

KeplerGl(data={'SOURCE': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, …

# 6. Generate Risk Scores

With these risk factors collected, we can now weight them and generate our final risk score.

Feel free to play around with different weights. Just make sure they sum to `1`

### Creating the Final Weighted Risk Score

Now we'll combine all our risk factors into a single composite risk score. First, we define weights for each factor, ensuring they sum to 1.0:

- Building count (15%): Number of structures at risk
- Building density (20%): Concentration of development
- Maximum burn probability (20%): Worst-case fire scenario
- Minimum burn probability (5%): Best-case fire scenario
- Range of burn probability (10%): Variability in fire risk
- Mean burn probability (10%): Average fire risk
- Distance to fire stations (20%): Emergency response capability

The SQL query performs several steps:
1. Joins fire station distances with our burn probability and building metrics
2. Calculates min/max values for each metric across all hexagons
3. Normalizes each metric to a 0-1 scale using (value - min)/(max - min)
4. Applies the weights to each normalized metric
5. Sums the weighted metrics for a final risk score between 0-1

The normalization ensures we can meaningfully combine metrics that are on different scales (like building counts and probabilities). Higher final scores indicate areas of greater overall wildfire risk.

We cache the results for efficient access in subsequent analysis and visualization steps.

In [22]:
%%time
weights = {
"norm_building_count" : .15,
"norm_building_density" : .2,
"norm_max_bp" : .2,
"norm_min_bp" : .05,
"norm_range_bp" : .1,
"norm_mean_bp" : .1,
"from_distance_ff" : .2
}

norm_df = sedona.sql(f"""
with max_burn_ff_dist AS (

    SELECT 
        t1.*,
        t2.DISTANCE
    FROM 
        max_burn t1
    JOIN
        fire_station_distance t2
    USING (h3)

),


 min_max AS (
    SELECT
       MIN(building_count) AS min_building_count,
        MAX(building_count) AS max_building_count,
        MIN(building_density) AS min_building_density,
        MAX(building_density) AS max_building_density,
        MIN(max_bp) AS min_max_bp,
        MAX(max_bp) AS max_max_bp,
        MIN(min_bp) AS min_min_bp,
        MAX(min_bp) AS max_min_bp,
        MIN(range_bp) AS min_range_bp,
        MAX(range_bp) AS max_range_bp,
        MIN(mean_bp) AS min_mean_bp,
        MAX(mean_bp) AS max_mean_bp,
        MIN(DISTANCE) AS min_dist_ff,
        MAX(DISTANCE) AS max_dist_ff
    FROM 
        max_burn_ff_dist),

normalized_data AS (
    SELECT
        t1.h3,
        t1.geometry,
        (t1.building_count - s.min_building_count) / (s.max_building_count - s.min_building_count) AS norm_building_count,
        (t1.building_density - s.min_building_density) /  (s.max_building_density - s.min_building_density) AS norm_building_density,
        (t1.max_bp - s.min_max_bp) /     (s.max_max_bp - s.min_max_bp) AS norm_max_bp,
        (t1.min_bp - s.min_min_bp) /     (s.max_min_bp - s.min_min_bp) AS norm_min_bp,
        (t1.range_bp - s.min_range_bp) / (s.max_range_bp - s.min_range_bp) AS norm_range_bp,
        (t1.mean_bp - s.min_mean_bp) /  (s.max_mean_bp - s.min_mean_bp) AS norm_mean_bp,
        (t1.DISTANCE - s.min_dist_ff) /  (s.max_dist_ff - s.min_dist_ff) AS norm_distance_bp
    FROM
        max_burn_ff_dist t1, MIN_MAX s
        )
-- Now add the weights to the normalized columns
SELECT
    h3,
    geometry,
    norm_building_count * {weights['norm_building_count']} AS weighted_norm_building_count,
    COALESCE(norm_building_density,0) * {weights['norm_building_density']}  AS weighted_norm_building_density,
    norm_max_bp * {weights['norm_max_bp']}  AS weighted_norm_max_bp,
    norm_min_bp * {weights['norm_min_bp']}  AS weighted_norm_min_bp,
    norm_range_bp * {weights['norm_range_bp']}  AS weighted_norm_range_bp,
    norm_mean_bp * {weights['norm_mean_bp']}  AS weighted_norm_mean_bp,
    norm_distance_bp * {weights['from_distance_ff']}  AS weighted_norm_distance_bp,
    -- Optional: Aggregate all weighted scores for a final score if needed
    (norm_building_count * {weights['norm_building_count']}  + 
     norm_building_density * {weights['norm_building_density']}  + 
     norm_max_bp * {weights['norm_max_bp']}  + 
     norm_min_bp * {weights['norm_min_bp']}  + 
     norm_range_bp * {weights['norm_range_bp']} +
     norm_mean_bp * {weights['norm_mean_bp']} +
     norm_distance_bp * {weights['from_distance_ff']} ) AS total_weighted_score
FROM normalized_data

""")
norm_df =norm_df.cache()

CPU times: user 1.66 ms, sys: 260 μs, total: 1.92 ms
Wall time: 189 ms


In [23]:
norm_df.select("h3","total_weighted_score").show(10)

+------------------+--------------------+
|                h3|total_weighted_score|
+------------------+--------------------+
|604215074038480895| 0.08626870280455891|
|604214927875375103| 0.24640960064895798|
|604214717958848511|  0.0804916086065132|
|604214681988497407|  0.1272767842400594|
|604214797415743487| 0.02530707364203897|
|604755131681996799|0.030181474615308256|
|604214988541788159| 0.04714184254956568|
|604215048671330303|0.031570527827502554|
|604755132621520895|  0.0556016137598665|
|604214745473482751|  0.1416786678167719|
+------------------+--------------------+
only showing top 10 rows



In [24]:
max_burn_df.unpersist()

DataFrame[h3: bigint, geometry: udt, building_count: bigint, building_density: double, max_bp: double, min_bp: double, mean_bp: double, range_bp: double]

### Make a map of the normalized risk factors

Create a choropleth map of the differetn risk factors and final risk score. Do any of the engineered features show a pattern?

In [25]:
tiledMap = SedonaKepler.create_map()
SedonaKepler.add_df(tiledMap, norm_df, name="Weighted Score")
tiledMap

KeplerGl(data={'Weighted Score': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, …

In [26]:
norm_df.printSchema()

root
 |-- h3: long (nullable = true)
 |-- geometry: geometry (nullable = true)
 |-- weighted_norm_building_count: double (nullable = true)
 |-- weighted_norm_building_density: double (nullable = false)
 |-- weighted_norm_max_bp: double (nullable = true)
 |-- weighted_norm_min_bp: double (nullable = true)
 |-- weighted_norm_range_bp: double (nullable = true)
 |-- weighted_norm_mean_bp: double (nullable = true)
 |-- weighted_norm_distance_bp: double (nullable = true)
 |-- total_weighted_score: double (nullable = true)



# 7. Examine Risk factors for Clusters

Getis-Ord  local Hot Spot analysis

https://docs.wherobots.com/latest/tutorials/wherobotsai/hotspot-detection/getis_ord/?h=geti

### Hot Spot Analysis Using Getis-Ord Statistics

After calculating our risk scores, we'll perform a hot spot analysis to identify statistically significant clusters of high-risk and low-risk areas. We use the Getis-Ord Gi* statistic, which tells us whether high or low values cluster together spatially.

The analysis involves:
1. Creating a spatial weights matrix:
  - distance_band_radius = 0.01 degrees (approximately 1.1km)
  - Binary weighting (1 if within radius, 0 if not)
  - Including each hexagon in its own calculation

2. Running Getis-Ord Gi* on our weighted_norm_distance_bp metric
  - Positive Z-scores: Clusters of high values (hot spots)
  - Negative Z-scores: Clusters of low values (cold spots)
  - P-values tell us the statistical significance

The results include:
- G: The Gi* statistic
- EG: Expected value of G
- VG: Variance of G
- Z: Z-score (how many standard deviations from the mean)
- P: P-value (statistical significance)

Try running this analysis on different risk factors by changing the metric after "weighted_norm_" in the g_local function!

In [27]:
%%time
#Wall time: 9.39 s
from sedona.stats.hotspot_detection.getis_ord import g_local
from sedona.stats.weighting import add_binary_distance_band_column
from sedona.stats.weighting import add_distance_band_column

distance_band_radius = .01

gi_df = g_local(
    add_distance_band_column( # adds the "weights" columns
        norm_df,
        distance_band_radius,
        binary=True,
        include_self=True,
        self_weight=1.0
    ),
    "weighted_norm_distance_bp", ## <--------- TEST DIFFERENT RISK FACTORS HERE
    "weights",
    star=True
).drop("weights")\
.drop("h3CellGeom")

<timed exec>:2: DeprecationWarning: Importing from 'sedona.stats.hotspot_detection.getis_ord' is deprecated. Please use 'sedona.spark.stats.hotspot_detection.getis_ord' instead.
<timed exec>:3: DeprecationWarning: Importing from 'sedona.stats.weighting' is deprecated. Please use 'sedona.spark.stats.weighting' instead.


CPU times: user 0 ns, sys: 8.86 ms, total: 8.86 ms
Wall time: 3.61 s


In [28]:
gi_df.show(2)

+------------------+--------------------+----------------------------+------------------------------+--------------------+--------------------+----------------------+---------------------+-------------------------+--------------------+--------------------+--------------------+--------------------+-------------------+--------------------+
|                h3|            geometry|weighted_norm_building_count|weighted_norm_building_density|weighted_norm_max_bp|weighted_norm_min_bp|weighted_norm_range_bp|weighted_norm_mean_bp|weighted_norm_distance_bp|total_weighted_score|                   G|                  EG|                  VG|                  Z|                   P|
+------------------+--------------------+----------------------------+------------------------------+--------------------+--------------------+----------------------+---------------------+-------------------------+--------------------+--------------------+--------------------+--------------------+-------------------+--

#### Make a map and symbolize by Z value and the P score
For the visual below you'll need to make some adjustments to get the best result:
- Change the tooltip to only show Z and P values
- Set the Fill Color to use a hot/cold color ramp based on the value of the Z-score

For interpreting the map:
- A positive Z score indicates a higher burn risk cluster
- A negative Z scores indicates a lower burn risk cluster
- The lower the P-score, the more statistically valid a cluster is

In [29]:
tiledMap = SedonaKepler.create_map()
SedonaKepler.add_df(tiledMap, gi_df, name="Weighted Score")
tiledMap

KeplerGl(data={'Weighted Score': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, …

In [30]:
print(f"Context Start Time: {(analysis_start-start)/60}")
print(f"Analysis Time: {(datetime.now()-analysis_start)/60}")
print(f"Total Time: {((analysis_start-start)/60) + ((datetime.now()-analysis_start)/60)}")


Context Start Time: 0:00:00.027031
Analysis Time: 0:00:01.769897
Total Time: 0:00:01.796929


## References
- https://wildfirerisk.org/download/
- https://www.fs.usda.gov/rds/archive/catalog/RDS-2020-0016-2